# Cliopatria viewer

1. To get started, download a copy of the Cliopatria dataset from here: `[INSERT LINK]`
2. `(OPTIONAL)` You can also download the GADM modern borders dataset from here `[INSERT LINK]`
3. Move the downloaded shape dataset(s) to an appropriate location on your machine and pass in the paths in the code cell below and run
4. Run the subsequent cells of the notebook


In [2]:
from map_functions import cliopatria_gdf, create_map
import folium
import ipywidgets as widgets
from IPython.display import display

In [3]:
# cliopatria_path = "path/to/cliopatria_dir/"
# gadm_path = "path/to/gadm_410.gpkg"
cliopatria_geojson_path = "../data/cliopatria_05192024/cliopatria.geojson"
cliopatria_json_path = "../data/cliopatria_05192024/name_years.json"
gadm_path = "../data/gadm_410.gpkg"

In [4]:
# Load the GADM geopackage
# gadm = gpd.read_file(gadm_path)
# print(gadm.head())

In [5]:
# gadm_countries = gadm.dissolve(by="COUNTRY").reset_index()
# gadm_countries['geom'] = gadm_countries['geom'].simplify(0.01)

In [6]:
# Load the Cliopatria data to a GeoDataFrame including end years for each shape
gdf = cliopatria_gdf(cliopatria_geojson_path, cliopatria_json_path)
print(gdf.head())

                             Name  Year      Area_km2    Type  \
0            Sumerian City-States -3400  22083.609657  POLITY   
1            Sumerian City-States -3200  35508.841506  POLITY   
2                            Elam -3200   4919.440675  POLITY   
3            Sumerian City-States -3000  45135.556672  POLITY   
4  Early Dynastic Period of Egypt -3000  92480.979261  POLITY   

                       Wikipedia     Color      SeshatID Member_of Components  \
0               History of Sumer  0x800000                                      
1               History of Sumer  0x800000                                      
2                           Elam  0xd9b800                                      
3               History of Sumer  0x800000                                      
4  Early Dynastic Period (Egypt)  0x808000  eg_dynasty_1                        

                                            geometry EndYear  
0  POLYGON ((46.58681 31.27192, 46.43482 31.27192...   -320

In [9]:
# Initial year to display
display_year = 0

# Create a text box for input
year_input = widgets.IntText(
    value=display_year,
    description='Year:',
)

# Define a function to be called when the value of the text box changes
def on_value_change(change):
    create_map(change['new'], gdf, map_output)

# Create a slider for input
year_slider = widgets.IntSlider(
    value=display_year,
    min=gdf['Year'].min(),
    max=gdf['EndYear'].max(),
    description='Year:',
)

# Link the text box and the slider
widgets.jslink((year_input, 'value'), (year_slider, 'value'))

# Create an output widget
map_output = widgets.Output()

# Create play and pause buttons
play_button = widgets.Button(description="Play")
pause_button = widgets.Button(description="Pause")

# Define the function to be called when the play button is clicked
def on_play_button_clicked(b):
    global is_playing
    is_playing = True
    while is_playing and year_slider.value < year_slider.max:
        year_slider.value += 1
        create_map(year_slider.value, gdf, map_output)  # Update the map after changing the slider value
        if not is_playing:
            break
        time.sleep(0.5)
        if not is_playing:
            break

# Define the function to be called when the pause button is clicked
def on_pause_button_clicked(b):
    global is_playing
    is_playing = False

# Create a boolean variable to control the play and pause state
is_playing = False

# Create a folium map centered at the center of the world
m = folium.Map(location=[0, 0], zoom_start=2, tiles='https://a.basemaps.cartocdn.com/rastertiles/voyager_nolabels/{z}/{x}/{y}.png', attr='CartoDB')

# Attach the function to the text box
year_input.observe(on_value_change, names='value')

# Attach the function to the play button
play_button.on_click(on_play_button_clicked)

# Attach the function to the pause button
pause_button.on_click(on_pause_button_clicked)

# Display the widgets
display(year_input, year_slider, play_button, pause_button, map_output)

# Create the map with the default year
create_map(display_year, gdf, map_output)

IntText(value=0, description='Year:')

IntSlider(value=0, description='Year:', max=2024, min=-3400)

Button(description='Play', style=ButtonStyle())

Button(description='Pause', style=ButtonStyle())

Output()